In [1]:
!pip install -q pennylane pennylane-lightning torchvision torch scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 935.6/935.6 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 12.5 MB/s eta 0:00:00


In [2]:
import os, shutil, random, json
import numpy as np
from time import time
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix
)

import pennylane as qml

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)


Device: cpu


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
DATA_DIR_RUN   = "/content/chest_xray"
DATA_DIR_DRIVE = "/content/drive/MyDrive/chest_xray"  # optional drive path

# Copy from "Drive if exists
if os.path.exists(DATA_DIR_DRIVE) and not os.path.exists(DATA_DIR_RUN):
    shutil.copytree(DATA_DIR_DRIVE, DATA_DIR_RUN)
    print("Dataset copied from Drive.")
else:

    print("Dataset already present.")

assert os.path.exists(DATA_DIR_RUN), "Dataset folder not found!"


Dataset copied from Drive.


In [5]:
INPUT_SIZE = 64
BATCH_SIZE = 16

train_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.15, contrast=0.15),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.ImageFolder(f"{DATA_DIR_RUN}/chest_xray/train", train_transform)
test_dataset  = datasets.ImageFolder(f"{DATA_DIR_RUN}/chest_xray/test",  test_transform)

targets = [label for _, label in train_dataset.samples]
class_counts = np.bincount(targets)
class_weights = 1. / torch.tensor(class_counts, dtype=torch.float)
sample_weights = [class_weights[label] for label in targets]

sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False)

print("Train samples:", len(train_dataset))
print("Test samples:", len(test_dataset))

Train samples: 5216
Test samples: 624


In [6]:
N_QUBITS = 8
N_QFEATURES = 6  # number of quantum outputs

try:
    dev = qml.device("lightning.qubit", wires=N_QUBITS)
    print("Quantum backend: lightning.qubit")
except:
    dev = qml.device("default.qubit", wires=N_QUBITS)
    print("Quantum backend: default.qubit")

def qconv_block(weights, wires):
    for i, w in enumerate(weights):
        qml.RY(w, wires=wires[i])
    for i in range(len(wires)-1):
        qml.CNOT([wires[i], wires[i+1]])
    qml.CNOT([wires[-1], wires[0]])

def qpool_block(weights, pairs):
    for j, (a, b) in enumerate(pairs):
        qml.RZ(weights[j], wires=a)
        qml.CNOT([a, b])

@qml.qnode(dev, interface="torch")
def qcnn(inputs, w_conv1, w_pool1, w_conv2, w_pool2):
    qml.AngleEmbedding(inputs, wires=range(N_QUBITS), rotation="Y")
    qconv_block(w_conv1, wires=list(range(N_QUBITS)))
    qpool_block(w_pool1, [(0,1),(2,3),(4,5),(6,7)])
    qconv_block(w_conv2, wires=[0,2,4,6])
    qpool_block(w_pool2, [(0,2),(4,6)])
    return [qml.expval(qml.PauliZ(i)) for i in [0,1,2,4,5,6]]


Quantum backend: lightning.qubit


In [7]:
weight_shapes = {
    "w_conv1": (N_QUBITS,),
    "w_pool1": (N_QUBITS//2,),
    "w_conv2": (N_QUBITS//2,),
    "w_pool2": (N_QUBITS//4,),
}

QLayer = qml.qnn.TorchLayer(qcnn, weight_shapes)


In [8]:
class HybridQCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.feat = nn.Sequential(
            nn.Conv2d(1, 16, 3, 2, 1), nn.ReLU(),
            nn.Conv2d(16, 32, 3, 2, 1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, 2, 1), nn.ReLU(),
            nn.Dropout(0.25),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(64, N_QUBITS),
            nn.Tanh()
        )
        self.q = QLayer
        self.post = nn.Sequential(
            nn.Linear(N_QFEATURES, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        x = self.feat(x) * np.pi
        q_out = self.q(x.cpu()).to(DEVICE)
        return self.post(q_out).view(-1)

model = HybridQCNN().to(DEVICE)


In [9]:
class SmoothBCELoss(nn.Module):
    def __init__(self, smoothing=0.05):
        super().__init__()
        self.smoothing = smoothing
        self.bce = nn.BCEWithLogitsLoss()

    def forward(self, logits, targets):
        # apply label smoothing:
        # 1 → 1 - smoothing
        # 0 → smoothing
        smooth_targets = targets * (1 - self.smoothing) + 0.5 * self.smoothing
        return self.bce(logits, smooth_targets)

criterion = SmoothBCELoss(smoothing=0.05)


In [10]:
# === LOSS ===
class SmoothBCELoss(nn.Module):
    def __init__(self, smoothing=0.05):
        super(SmoothBCELoss, self).__init__()
        self.smoothing = smoothing
        self.bce = nn.BCEWithLogitsLoss()

    def forward(self, logits, targets):
        smooth_targets = targets * (1 - self.smoothing) + 0.5 * self.smoothing
        return self.bce(logits, smooth_targets)

criterion = SmoothBCELoss(smoothing=0.05)

# === OPTIMIZER ===
optimizer = optim.AdamW(model.parameters(), lr=7e-4, weight_decay=1e-4)


# PyTorch (Colab version) does NOT support verbose=True
scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=10,
    eta_min=1e-5
    # mode="min",
    # patience=2,
    # factor=0.5
)


In [11]:
def train_epoch():
    model.train()
    total_loss = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(DEVICE), labels.float().to(DEVICE)
        optimizer.zero_grad()
        logits = model(imgs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(train_loader)


In [12]:
@torch.no_grad()
def evaluate():
    model.eval()
    all_probs, all_labels = [], []

    for imgs, labels in test_loader:
        imgs = imgs.to(DEVICE)
        logits = model(imgs)
        probs = torch.sigmoid(logits).cpu().numpy()
        all_probs.extend(probs)
        all_labels.extend(labels.numpy())

    preds = (np.array(all_probs) > 0.5).astype(int)
    labels = np.array(all_labels)

    return {
        "accuracy": accuracy_score(labels, preds) * 100,
        "precision": precision_score(labels, preds),
        "recall": recall_score(labels, preds),
        "f1": f1_score(labels, preds),
        "auc": roc_auc_score(labels, all_probs),
        "confusion": confusion_matrix(labels, preds)
    }


In [13]:
EPOCHS = 12
for epoch in range(1, EPOCHS+1):
    loss = train_epoch()
    print(f"Epoch {epoch}/{EPOCHS} - Loss: {loss:.4f}")
    scheduler.step()
print("Training completed!")`

Epoch 1/12 - Loss: 0.6025
Epoch 2/12 - Loss: 0.4158
Epoch 3/12 - Loss: 0.3474
Epoch 4/12 - Loss: 0.3323
Epoch 5/12 - Loss: 0.3152
Epoch 6/12 - Loss: 0.3017
Epoch 7/12 - Loss: 0.2922
Epoch 8/12 - Loss: 0.2818
Epoch 9/12 - Loss: 0.2816
Epoch 10/12 - Loss: 0.2653
Epoch 11/12 - Loss: 0.2673
Epoch 12/12 - Loss: 0.2593
Training completed!


In [14]:
metrics = evaluate()

for k, v in metrics.items():
    if k != "confusion":
        print(f"{k}: {v}")

print("Confusion Matrix:\n", metrics["confusion"])

accuracy: 84.13461538461539
precision: 0.8523002421307506
recall: 0.9025641025641026
f1: 0.8767123287671232
auc: 0.9180254218715757
Confusion Matrix:
 [[173  61]
 [ 38 352]]


In [15]:
import numpy as np
import json

# Convert numpy types → Python types
safe_metrics = {}

for k, v in metrics.items():
    if isinstance(v, np.ndarray):
        safe_metrics[k] = v.tolist()          # convert array → list
    elif isinstance(v, (np.float32, np.float64)):
        safe_metrics[k] = float(v)            # convert numpy float → python float
    else:
        safe_metrics[k] = v                   # leave normal python data types

# Save safely
with open("/content/hybrid_qcnn_metrics.json", "w") as f:
    json.dump(safe_metrics, f, indent=4)

print("Metrics saved successfully!")



Metrics saved successfully!


In [16]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.4 MB/s eta 0:00:00


In [17]:
!pip install pyngrok


In [ ]:
!ngrok config add-authtoken <add authtoken of yours>

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [19]:
%%writefile app.py
# paste your Streamlit code here
import os
import json
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image, ImageOps
import numpy as np
import cv2
import streamlit as st
import pennylane as qml


# ===========================================================
#                MODEL PATHS (CORRECT)
# ===========================================================

MODEL_PATH = "/content/drive/MyDrive/hybrid_qcnn.pth"
METRICS_PATH = "/content/drive/MyDrive/hybrid_qcnn_metrics.json"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# ===========================================================
#                QUANTUM QCNN CIRCUIT
# ===========================================================

n_qubits = 8
dev = qml.device("default.qubit", wires=n_qubits)

def qconv_block(weights, wires):
    for i, w in enumerate(weights):
        qml.RY(w, wires=wires[i])
    for i in range(len(wires)):
        qml.CNOT(wires=[wires[i], wires[(i+1)%len(wires)]])

def qpool_block(weights, pairs):
    for j, (a,b) in enumerate(pairs):
        qml.RZ(weights[j], wires=a)
        qml.CNOT(wires=[a,b])

@qml.qnode(dev, interface="torch")
def qcnn_circuit(inputs, w_conv1, w_pool1, w_conv2, w_pool2):
    qml.AngleEmbedding(inputs, wires=range(n_qubits), rotation="Y")
    qconv_block(w_conv1, wires=list(range(n_qubits)))
    qpool_block(w_pool1, [(0,1),(2,3),(4,5),(6,7)])
    qconv_block(w_conv2, wires=[0,2,4,6])
    qpool_block(w_pool2, [(0,2),(4,6)])
    return qml.expval(qml.PauliZ(0))

weight_shapes = {
    "w_conv1": (n_qubits,),
    "w_pool1": (n_qubits//2,),
    "w_conv2": (n_qubits//2,),
    "w_pool2": (n_qubits//4,),
}

QLayer = qml.qnn.TorchLayer(qcnn_circuit, weight_shapes)


# ===========================================================
#       ⭐ EXACT TRAINED MODEL ARCHITECTURE (MATCHING WEIGHTS)
# ===========================================================

class HybridQCNN(nn.Module):
    def __init__(self):
        super().__init__()

        # Your original architecture (from checkpoint)
        self.feat = nn.Sequential(
            nn.Conv2d(1, 8, 3, 2, 1), nn.ReLU(inplace=True),     # 1 → 8
            nn.Conv2d(8, 16, 3, 2, 1), nn.ReLU(inplace=True),    # 8 → 16
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(16, 8),                                    # 16 → 8
            nn.Tanh()
        )

        self.q = QLayer

        self.post = nn.Sequential(
            nn.Linear(1, 16),   # 1 → 16
            nn.ReLU(),
            nn.Linear(16, 1)    # 16 → 1 (logit)
        )

    def forward(self, x):
        x = self.feat(x) * torch.pi
        q_out = self.q(x).unsqueeze(1)      # (B,1)
        return self.post(q_out).view(-1)    # logits


# ===========================================================
#                LOAD TRAINED MODEL & METRICS
# ===========================================================

model = HybridQCNN().to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

accuracy = None
if os.path.exists(METRICS_PATH):
    with open(METRICS_PATH, "r") as f:
        accuracy = json.load(f).get("test_accuracy", None)


# ===========================================================
#              🌟 IMPROVED X-RAY VALIDATOR
# ===========================================================

def is_xray_image(pil_img):
    """
    More tolerant but still blocks obviously invalid images.
    """
    img_np = np.array(pil_img)
    gray = np.array(ImageOps.grayscale(pil_img))

    # 1) Color check
    if len(img_np.shape) == 3:
        b, g, r = img_np[:,:,0], img_np[:,:,1], img_np[:,:,2]
        color_diff = np.mean(np.abs(r - g)) + np.mean(np.abs(g - b))
        if color_diff > 80:      # relaxed threshold
            return False

    # 2) Edge density check
    edges = cv2.Canny(gray, 20, 80)
    edge_density = np.sum(edges > 0) / edges.size
    if edge_density < 0.003:
        return False

    # 3) Brightness check
    brightness = np.mean(gray)
    if brightness < 5 or brightness > 245:
        return False

    return True


# ===========================================================
#                PREPROCESS + PREDICT
# ===========================================================

test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

def predict(pil_img):
    img = test_transform(pil_img).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(img)
        prob = torch.sigmoid(logits).item()
    return ("Pneumonia" if prob > 0.5 else "Normal"), prob


# ===========================================================
#                HEALTH SUGGESTIONS
# ===========================================================

def get_suggestions(label):

    if label == "Normal":
        return """
        ### 🟢 Suggestions for Normal Lungs
        - Maintain regular exercise
        - Avoid smoking/pollution
        - Stay hydrated and eat nutritious food
        - Practice breathing exercises
        - Get annual lung check-ups
        """

    return """
    ### 🔴 Precautions for Pneumonia
    - Consult a doctor immediately
    - Take prescribed antibiotics
    - Keep yourself warm
    - Drink warm fluids
    - Do steam inhalation 2–3 times daily
    - Rest well and avoid physical strain
    - Monitor oxygen levels regularly
    """


# ===========================================================
#                ⭐ STREAMLIT UI (Modern & Clean)
# ===========================================================

st.set_page_config(page_title="Quantum CNN Pneumonia Detector", page_icon="🩺", layout="centered")

st.markdown("""
    <h1 style='text-align:center; color:#2c3e50;'>🩺 Quantum CNN — Pneumonia Detector</h1>
    <p style='text-align:center;'>
        Upload a chest X-ray image to detect <b>Normal</b> or <b>Pneumonia</b>.
    </p>
""", unsafe_allow_html=True)

if accuracy:
    st.info(f"Model Test Accuracy: **{84}%**")

uploaded = st.file_uploader("Upload Chest X-ray Image", type=["jpg","jpeg","png"])

if uploaded:
    img = Image.open(uploaded)
    st.image(img, caption="Uploaded X-ray", use_column_width=True)

    # Validate image
    if not is_xray_image(img):
        st.error("❌ This does NOT appear to be a chest X-ray image.")
        st.warning("Please upload a valid chest X-ray.")
    else:
        label, prob = predict(img)

        st.markdown(f"""
            <div style='background-color:#ecf0f1; padding:20px; border-radius:10px; margin-top:20px;'>
                <h3 style='text-align:center;'>🔍 Prediction Result</h3>
                <h2 style='text-align:center; color:#2980b9;'>{label}</h2>
                <p style='text-align:center;'>Confidence: <b>{prob:.4f}</b></p>
            </div>
        """, unsafe_allow_html=True)

        # Display suggestions
        st.markdown(get_suggestions(label))


Writing app.py


In [22]:
!pkill -f ngrok


In [23]:
from pyngrok import ngrok



# Create new public URL
public_url = ngrok.connect(8501)
print("PUBLIC URL:", public_url)

# Start Streamlit server
!streamlit run app.py --server.port 8501 &>/dev/null&


PUBLIC URL: NgrokTunnel: "https://376bd09d7b04.ngrok-free.app" -> "http://localhost:8501"
